In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from component.model import BenefitModel
from component.model import ConstraintModel
from component.model import CostModel
from component.model import SeplanAoi

from component.scripts.seplan import Seplan


from sepal_ui.mapping import SepalMap, InspectorControl
from component.scripts.seplan import _percentile

In [4]:
seplan_aoi = SeplanAoi(aoi_model=None)
benefit_model = BenefitModel()
constraint_model = ConstraintModel()
cost_model = CostModel()
seplan_model = Seplan(
    seplan_aoi,
    benefit_model,
    constraint_model,
    cost_model,
)

In [5]:
map_ = SepalMap()
inspector_control = InspectorControl(map_)
map_.add(inspector_control)

SepalMap(center=[0, 0], controls=(ZoomControl(options=['position', 'transparent_bg'], position='topright', wid…

In [6]:
import ee

ee.Initialize()
aoi = seplan_aoi.feature_collection

In [32]:
# Create a random point over the AOI=name=COL_Risaralda
random_point = {
    "type": "FeatureCollection",
    "columns": {},
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [-75.92060223140517, 4.861788659634801],
            },
            "id": "0",
            "properties": {},
        }
    ],
}


random_point = ee.FeatureCollection(random_point).first().geometry()
# Now let's extract the value of the normalize benefits

## Create a test dataset with a random value from each benefit


In [24]:
# Okay I'm not using GEE functions because I think that the performance would
# be more or less the same, as we are not using any getinfo function, so at
# the end we are just building the graph.

In [37]:
perc_images = [
    _percentile(ee.Image(i).unmask(), aoi) for i in seplan_model.benefit_model.assets
]

# create a list of images with only one value (the value of the random point)
images = []
for image in perc_images:
    images.append(
        ee.Image(
            ee.Number(
                image.reduceRegion(
                    reducer=ee.Reducer.first(),
                    geometry=random_point,
                    scale=image.projection().nominalScale(),
                )
                .values()
                .get(0)
            )
        )
    )

In [96]:
# Those values are the following
[
    0.10720205307006836,
    0.03448275849223137,
    0.9781022071838379,
    0.9337244629859924,
    0.964881181716919,
    0.9337817430496216,
]

[0.10720205307006836,
 0.03448275849223137,
 0.9781022071838379,
 0.9337244629859924,
 0.964881181716919,
 0.9337817430496216]

In [56]:
# Get the value by using the min value of the image (min max are the same)
image_values = [image.getInfo()["bands"][0]["data_type"]["min"] for image in images]

In [97]:
self.benefit_model.themes

['bii', 'bii', 'carbon_seq', 'local', 'local', 'wood']

In [94]:
# Test the function that is used

self = seplan_model

# Create an empty dictionary with the themes as keys
default = {"image": ee.Image(0), "weight": 0, "nb": 0}
theme_images = {k: default.copy() for k in set(self.benefit_model.themes)}

for idx, image in enumerate(images):
    #  Get the theme of the image by the index of the asset
    theme_image = theme_images[self.benefit_model.themes[idx]]
    theme_image["image"] = theme_image["image"].add(image)
    theme_image["weight"] += self.benefit_model.weights[idx]
    theme_image["nb"] += 1

for v in theme_images.values():
    v["weight"] = v["weight"] / v["nb"]
    v["image"] = v["image"].divide(v["nb"])

total_weight = sum([v["weight"] for v in theme_images.values()])

index = ee.Image(0)
for v in theme_images.values():
    index = index.add(v["image"].multiply(ee.Image(v["weight"]).divide(total_weight)))

In [95]:
index.getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0.7330072945915163,
    'max': 0.7330072945915163},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [81]:
theme_images.keys()

dict_keys(['wood', 'local', 'bii', 'carbon_seq'])

In [82]:
test_theme = theme_images["wood"]
test_theme

{'image': <ee.image.Image at 0x7f601da519f0>, 'weight': 4.0, 'nb': 1}

In [88]:
test_theme["image"].getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0.9337817430496216,
    'max': 0.9337817430496216},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [89]:
0.9337817430496216 * 4

3.7351269721984863

In [85]:
test_theme["weight"]

4.0

In [83]:
test_theme["image"].multiply(ee.Image(test_theme["weight"])).getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 3.7351269721984863,
    'max': 3.7351269721984863},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [68]:
index.getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0.24661651143833296,
    'max': 0.24661651143833296},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [63]:
index.reduceRegion(
    reducer=ee.Reducer.first(),
    scale=index.projection().nominalScale(),
    geometry=random_point,
).getInfo()

{'constant': 0.24661651143833296}

In [61]:
theme_images

{'wood': {'image': <ee.image.Image at 0x7f601da52680>, 'weight': 4.0, 'nb': 1},
 'local': {'image': <ee.image.Image at 0x7f601da52770>,
  'weight': 4.0,
  'nb': 2},
 'bii': {'image': <ee.image.Image at 0x7f601da52890>, 'weight': 4.0, 'nb': 2},
 'carbon_seq': {'image': <ee.image.Image at 0x7f601da52830>,
  'weight': 4.0,
  'nb': 1}}

In [57]:
image_values

[0.10720205307006836,
 0.03448275849223137,
 0.9781022071838379,
 0.9337244629859924,
 0.964881181716919,
 0.9337817430496216]

In [58]:
image_weights = seplan_model.benefit_model.weights
image_weights

[4, 4, 4, 4, 4, 4]

In [59]:
image_themes = seplan_model.benefit_model.themes
image_themes

['bii', 'bii', 'carbon_seq', 'local', 'local', 'wood']

In [64]:
WAa = image_weights[0]
WAb = image_weights[1]
Aa = image_values[0]
Ab = image_values[1]
Wb = image_weights[2]
B = image_values[2]
WCa = image_weights[3]
WCb = image_weights[4]
Ca = image_values[3]
Cb = image_values[4]
Wd = image_weights[5]
D = image_values[5]

WA = (WAa + WAb) / 2
A = (Aa + Ab) / 2

WC = (WCa + WCb) / 2
C = (Ca + Cb) / 2

index = (WA * A + Wb * B + WC * C + Wd * D) / (WA + Wb + WC + Wd)
index

0.7330072945915163

In [93]:
(Wd * D) / (WA + Wb + WC + Wd)

0.2334454357624054